<a href="https://colab.research.google.com/github/hikmatfarhat-ndu/veronica-thesis/blob/master/malware.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Packages

In [1]:
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt
from tensorflow.keras import models,layers
from tensorflow.keras.utils import Sequence
#from tensorflow.python.keras.utils import data_utils
import math
import os
import pandas
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv3D,Conv2D,InputLayer,MaxPooling3D,Dropout,Flatten,MaxPooling2D,BatchNormalization,LayerNormalization


In [2]:
%%bash
fileid="1fjB9yNDIlRMm2Y4v2Ta2N60yuDBN2D7r&export=download" 
filename="resized.7z"
curl -L -c cookies.txt 'https://docs.google.com/uc?export=download&id='$fileid | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt

curl -L -b cookies.txt -o $filename 'https://docs.google.com/uc?export=download&id='$fileid'&confirm='$(<confirm.txt)

rm -f confirm.txt cookies.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3290    0  3290    0     0  13212      0 --:--:-- --:--:-- --:--:-- 13212
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   2103      0 --:--:-- --:--:-- --:--:--  2103
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  122M    0  122M    0     0  53.9M      0 --:--:--  0:00:02 --:--:-- 78.8M


In [3]:
!apt-get -y install p7zip-full

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  p7zip
Suggested packages:
  p7zip-rar
The following NEW packages will be installed:
  p7zip p7zip-full
0 upgraded, 2 newly installed, 0 to remove and 19 not upgraded.
Need to get 1522 kB of archives.
After this operation, 5755 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 p7zip amd64 16.02+dfsg-6 [358 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 p7zip-full amd64 16.02+dfsg-6 [1164 kB]
Fetched 1522 kB in 1s (1057 kB/s)   
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package p7zip.
(Reading database ... 100757 files and directories currently installed.)
Preparing to unpack .../p7zip_16.02+dfsg-6_amd64.deb ...
Unpacking p7zip (16.02+dfsg-6) ...
Selecting previously unselected package p7zip-full.
Preparing

In [4]:
!rm -rf test train
!7z x resized.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=C.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Sca        1 file, 128902383 bytes (123 MiB)

Extracting archive: resized.7z
--
Path = resized.7z
Type = 7z
Physical Size = 128902383
Headers Size = 269314
Method = LZMA2:24
Solid = +
Blocks = 1

      4% 492 - test/Kelihos_ver3/eL9bqR8yt05jTBWgVnFs.pn                                                     11% 1095 - test/Lollipop/k8yohOL7S2sewB9Wzfp5.p                                                 15% 1399 - test/Ramnit/Ej1OaeJm6GkZ7oQ4pTUw.p                                               18% 1808 - test/Vundo/L5MO1uWhDZ2yplTkoRUd.pn                                               27% 2477 - train/Gatak/a1u9v0bUgGqk5nBwQ2tm.p                                               31% 2692 - train/Gatak/hFAlObnD1iHQTdgMGBY2.p                                              

## Read the data from directory

The data are the original code converted to images and then resized to 256x256ove sets.

In [5]:
image_size=(256,256)
dir="./"
dataset=tf.keras.preprocessing.image_dataset_from_directory(dir+"train",batch_size=32,image_size=image_size,
                                                            shuffle=True,color_mode="grayscale")
test_dataset=tf.keras.preprocessing.image_dataset_from_directory(dir+"test",batch_size=32,image_size=image_size,
                                                            shuffle=False,color_mode="grayscale")

Found 9000 files belonging to 9 classes.
Found 1868 files belonging to 9 classes.


## Build the model

In [6]:
def createModelYuan():
    
    model = Sequential()
    model.add(InputLayer(input_shape=(256,256,1)))
    #model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1./255))
   
    model.add(Conv2D(64, kernel_size = (3,3),strides=(1,1), padding='same',activation = 'relu'))
    model.add(Conv2D(64, kernel_size = (3,3),strides=(1,1), padding='same',activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Conv2D(128, kernel_size = (3,3),strides=(1,1), padding='same',activation = 'relu'))
    model.add(Conv2D(128, kernel_size = (3,3),strides=(1,1), padding='same',activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))
    

    model.add(Conv2D(256, kernel_size = (3,3),strides=(1,1), padding='same',activation = 'relu'))
    model.add(Conv2D(256, kernel_size = (3,3), strides=(1,1),padding='same',activation = 'relu'))
    model.add(Conv2D(256, kernel_size = (3,3), strides=(1,1),padding='same',activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))


    model.add(Conv2D(512, kernel_size = (3,3), strides=(1,1),padding='same',activation = 'relu'))
    model.add(Conv2D(512, kernel_size = (3,3), strides=(1,1),padding='same',activation = 'relu'))
    model.add(Conv2D(512, kernel_size = (3,3), strides=(1,1),padding='same',activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Conv2D(512, kernel_size = (3,3), strides=(1,1),padding='same',activation = 'relu'))
    model.add(Conv2D(512, kernel_size = (3,3), strides=(1,1),padding='same',activation = 'relu'))
    model.add(Conv2D(512, kernel_size = (3,3), strides=(1,1),padding='same',activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))


    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(9, activation = 'softmax', name = 'Output'))
    return model


### Instantiate the model

In [7]:

model=createModelYuan()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 64)      640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 64, 256)       2

## Optimization

Keras can use many optimization method. In this notebook we use the __Adam__ method which can be described loosely as __adaptive__ gradient descent.

Also since the labels are __NOT__ in one_hot_encoding we use the "Sparse" version of the crossentropy loss: __SparseCategoricalCrossentropy__. Finally, if we don't specify from_logits=False then the loss function would compute softwmax before computing the loss. Since we are computing softwmax in our model already we turn this step off by specifying from_logits=False

In [8]:
# if we don't use softmax in the last layer, i.e. if the output of the
# model is NOT probabilities then use from_logits=True

# where the weights are saved periodically 
filepath="checkpoints/cp-{epoch}.ckpt"
cb=tf.keras.callbacks.ModelCheckpoint(filepath,save_weights_only=True,save_freq='epoch',verbose=0)
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

latest = tf.train.latest_checkpoint("chec#kpoints")
#model.load_weights(latest)
history=model.fit(dataset,callbacks=[cb],epochs=30)


Epoch 1/30
  4/282 [..............................] - ETA: 21:40 - loss: 24.9309 - accuracy: 0.1562

KeyboardInterrupt: 

### Testing the Accuracy

In [ ]:

_,test_accuracy=model.evaluate(test_dataset)

In [ ]:
probabilities=model.predict(test_dataset)
predictions=np.argmax(probabilities,axis=1)


In [ ]:
y = np.concatenate([y for x, y in test_dataset], axis=0)


In [ ]:
m=tf.math.confusion_matrix(y,predictions)

In [ ]:
m

In [ ]:
import seaborn as sb

In [ ]:
m/sum(m)

In [ ]:
plt.figure(figsize=(10, 7))
sb.heatmap(m/sum(m), xticklabels=dataset.class_names, yticklabels=dataset.class_names, 
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.show()